In [ ]:
# imports
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

Create our LSTM class to make training LSTM much easier

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

class LSTMModel:
    def __init__(self, X_train, y_train, X_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.model = None

    def train(self, epochs=50, batch_size=32, verbose=1):
        # Reshape the input data for LSTM
        X_train = np.reshape(self.X_train, (self.X_train.shape[0], self.X_train.shape[1], 1))

        # Build the LSTM model
        self.model = Sequential()
        self.model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mse')

        # Train the model
        self.model.fit(X_train, self.y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

    def predict(self):
        # Reshape the input data for prediction
        X_test = np.reshape(self.X_test, (self.X_test.shape[0], self.X_test.shape[1], 1))

        # Make predictions
        predictions = self.model.predict(X_test)
        return predictions

Connect drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load train and test splits

In [ ]:


X_train =
y_train = ...
X_test = ...

Training the model

In [ ]:


# Initialize the LSTM model
lstm_model = LSTMModel(X_train, y_train, X_test)

# Train the model
lstm_model.train()

Testing model and graphing predictions

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'y_test' is your actual test target data
# Replace it with your actual test target data
y_test = ...

# Plot the actual test data
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual')

# Plot the predictions
plt.plot(predictions, label='Predicted')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Actual vs. Predicted')
plt.legend()
plt.show()
